In [ ]:
#load the dataset
!kaggle datasets download -d shrutisaxena/yoga-pose-image-classification-dataset
!unzip yoga-pose-image-classification-dataset.zip

In [ ]:
#for pose detection
import mediapipe as mp 
import numpy as np
#for image input output operations
import matplotlib.pyplot as plt
import matplotlib.image as pimg

mp_pose = mp.solutions.pose.Pose(min_detection_confidence=0.7,
                                 min_tracking_confidence=0.7)

# img = pimg.imread('dataset/upavistha konasana/2-0.png')
# if img.dtype != 'uint8':
#     img = (img * 255).astype('uint8')

# results = mp_pose.process(img)
# print(results)


I0000 00:00:1755281160.871183 2259491 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4


W0000 00:00:1755281160.934857 2588732 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755281160.943712 2588738 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
mp_drawing = mp.solutions.drawing_utils


mp_drawing.draw_landmarks(
    img,
    results.pose_landmarks,
    mp.solutions.pose.POSE_CONNECTIONS,
    mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=4, circle_radius=2),
    mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=6, circle_radius=2)

)
plt.imshow(img)

In [85]:
import tensorflow as tf


data_directory = 'dataset'

img_hieght = 224,
img_width = 224,
img_size = (224, 224)

batch_size = 32

training_dataset = tf.keras.utils.image_dataset_from_directory(
    data_directory, 
    validation_split = 0.3,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size = batch_size
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    validation_split=0.3,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)


val_batches = tf.data.experimental.cardinality(validation_dataset)
validation_dataset = validation_dataset.skip(val_batches//2)
test_dataset = validation_dataset.take(val_batches//2)




Found 5993 files belonging to 107 classes.
Using 4196 files for training.
Found 5993 files belonging to 107 classes.
Using 1797 files for validation.


Goal: Process the dataset to get keypoints data and store in a CSV for later processing.

In [ ]:
import pandas as pd
landmark_data = []

# images, labels = next(iter(training_dataset.take(1)))
# result = mp_pose.process(img)
# print(result.pose_landmarks.landmark)

#Extracting Key Point Data to add to csv file. 
key_points_list = []
labels_list = []
nan_count = 0
for images, labels in training_dataset:
    for i in range(images.shape[0]):
        img = images[i].numpy().astype('uint8')
        label = labels[i].numpy()
        result = mp_pose.process(img)
        if result.pose_landmarks:
            keypoints = []
            for lm in result.pose_landmarks.landmark:
                keypoints.extend([lm.x, lm.y, lm.z])
            key_points_list.append(keypoints)
            labels_list.append(label)
        else:
            nan_count +=1
            landmarks = np.zeros(33 * 3)
            key_points_list.append( landmarks)
            labels_list.append(label)

df = pd.DataFrame(key_points_list)
df['label'] = labels_list
df.to_csv("training_yoga1_keypoints.csv", index=False)


key_points_list = []
labels_list = []
for images, labels in validation_dataset:
    for i in range(images.shape[0]):
        img = images[i].numpy().astype('uint8')
        label = labels[i].numpy()
        result = mp_pose.process(img)
        if result.pose_landmarks:
            keypoints = []
            for lm in result.pose_landmarks.landmark:
                keypoints.extend([lm.x, lm.y, lm.z])
            key_points_list.append(keypoints)
            labels_list.append(label)
        else:
            nan_count +=1
            landmarks = np.zeros(33 * 3)
            key_points_list.append( landmarks)
            labels_list.append(label)

df = pd.DataFrame(key_points_list)
df['label'] = labels_list
df.to_csv("validation_yoga1_keypoints.csv", index=False)

key_points_list = []
labels_list = []
for images, labels in test_dataset:
    for i in range(images.shape[0]):
        img = images[i].numpy().astype('uint8')
        label = labels[i].numpy()
        result = mp_pose.process(img)
        if result.pose_landmarks:
            keypoints = []
            for lm in result.pose_landmarks.landmark:
                keypoints.extend([lm.x, lm.y, lm.z])
            key_points_list.append(keypoints)
            labels_list.append(label)
        else:
            nan_count +=1
            landmarks = np.zeros(33 * 3)
            key_points_list.append(landmarks)
            labels_list.append(label)
print(nan_count)
df = pd.DataFrame(key_points_list)
df['label'] = labels_list
df.to_csv("test_yoga1_keypoints.csv", index=False)




2025-08-17 17:40:05.657400: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile
2025-08-17 17:40:31.544403: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: cHRM: inconsistent chromaticities
2025-08-17 17:40:50.816118: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile
2025-08-17 17:40:53.434062: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile
2025-08-17 17:40:56.034721: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile
2025-08-17 17:41:08.134650: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile


1486
